In this notebook you will learn how to implement a Flow Matching Algorithm to create Animals, Note that you could change the Dataset and create different objects.

This is aswell Part of the project presented at ICRA 2026, made by @eduardoHernandez and @UlisesDiaz.

Each section will have a Mathematical Explation, the goal is to make it easier to understand. Thus, it will have several animations to make you understand

Note that this is animations and the mathematical explanation is not on the official Code.

We make this to help the community into this topic.

It is important to note that most of it, comes from : where, they explain each detail, so go and see it.


Hope you like it.


## 1. Install Dependencies

1.- Torch, torchvision, torchaudio,
2.- matplotlib , numpy, tqdm, pillow
3.- gdown
4.- einops

In [ ]:
!pip install torch torchvision torchaudio
!pip install matplotlib numpy tqdm pillow
!pip install gdown # for dataset download
!pip install einops  #for tensor manipulation

## 2. Initial Configuration

Standard Pytorch configurations


In [ ]:
# pytorch configurations

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.utils import make_grid, save_image


# python imports

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from tqdm import tqdm
from typing import Tuple, Optional
import math
import einops

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Seeds for reproducibility
'''
These are used to reproduce the experiment we have
'''

torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

cuda


## 3. Download and Prepare Dataset

Note that you can change for any dataset you want, the code should work the same

Download the dataset in: https://www.dropbox.com/scl/fi/kpxh6hu04eu28yb8l30wy/afhq.zip?e=2&rlkey=usjnva71u164xd4rq6ghlab1u&dl=0


## 4. Using the Dataset

Here,  we sample and get the transforms of the Dataset Above

In [ ]:
# First we need to pass from image to tensor

def get_transform(image_size = 64):
  return transforms.Compose(
      [
        transforms.Resize((image_size, image_size)),
        transform.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) #[R, G, B]
      ]
  )


# Create personalized Dataset

class AnimalDataset(Dataset) :
  def __init__(self, root_dir, transform = None) :
    self.root_dir = root_dir
    self.transform = transforms

    # Create list of images
    self.images = []

    # Search for any type of image

    for root, dirs, files in os.walk(root_dir):
      for file in files:
          if file.lower().endswith(('.png', '.jpg', '.jpeg')):
              self.images.append(os.path.join(root, file))


  def __len__(self) :
    return len(self.images)

  def __getitem__ (self, idx) :
    img_path = self.images[idx]
    image = Image.open(img_path).convert("RGB")

    if self.transform :
      image = self.transform(image)

    return image


IMAGE_SIZE = 64
BATHC_SIZE = 32

transforms = get_transform(IMAGE_SIZE)



tran_dataset = AnimalDataset('data/afhq/train', transform = transforms)

 # n_workers = how the data is process during training. Is like using a GPU, we have 4 workers loading data, applying transformations, and preparing batches
train_loader = DataLoader(train_dataset, batch_size = BATHC_SIZE, shuffle=True, num_workers=4)

print(f"Dataset cargado: {len(train_dataset)} imágenes")
print(f"Dimensiones de imagen: {IMAGE_SIZE}x{IMAGE_SIZE}")


def show_samples (loader, num_samples = 8) :
  batch = next(iter(loader))
  images = batch[:num_samples]

  images = (images + 1 )/2

  grid = make_grid (images, nrow = 4, padding= 2)
  plt.figure(figsize=(10,6))
  plt.imshow(grid.permute(1,2,0))
  plt.axis('off')
  plt.show()

show_samples(train_loader)




NameError: name 'Dataset' is not defined

# 5. Initial Gaussian Distribution

Here we start seeing actual stuff on Flow Matching.

Fist, we need a classs to manage the sampling of Gaussian Distribution x0 , we start from pure noise (Gaussian Distribution) to real data (images)


To see the explanation of Flow Matching and the Notes of the paper, see `section 0`

The explanation of this Part is in Lecture 1. Go and see it.


